In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


**Split train and test set**

In [3]:
df.drop(df.columns[[-1, 0]], axis=1, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [4]:
X = df[['radius_mean', 'perimeter_mean', 'area_mean', 'concavity_mean', 'concave points_mean']]
y = df[['diagnosis']]

In [5]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   diagnosis  569 non-null    object
dtypes: object(1)
memory usage: 4.6+ KB


In [6]:
yes_no_columns = ['diagnosis']
for col in yes_no_columns:
    y[col].replace({'B': 0,'M': 1},inplace=True)

C:\Users\gaura\Desktop\ANN\ann\lib\site-packages\pandas\core\generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [7]:
y.value_counts()

diagnosis
0            357
1            212
dtype: int64

In [8]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   diagnosis  569 non-null    int64
dtypes: int64(1)
memory usage: 4.6 KB


In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   radius_mean          569 non-null    float64
 1   perimeter_mean       569 non-null    float64
 2   area_mean            569 non-null    float64
 3   concavity_mean       569 non-null    float64
 4   concave points_mean  569 non-null    float64
dtypes: float64(5)
memory usage: 22.4 KB


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=25)

**Preprocessing: Scale the data so that both age and affordibility are in same scaling range**

In [11]:
X_train_scaled = X_train.copy()
X_train_scaled['radius_mean'] = X_train_scaled['radius_mean'] / 100  
X_train_scaled['perimeter_mean'] = X_train_scaled['perimeter_mean'] / 100
X_train_scaled['area_mean'] = X_train_scaled['area_mean'] / 100
X_train_scaled['concavity_mean'] = X_train_scaled['concavity_mean'] / 100
X_train_scaled['concave points_mean'] = X_train_scaled['concave points_mean'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['radius_mean'] = X_test_scaled['radius_mean'] / 100  
X_test_scaled['perimeter_mean'] = X_test_scaled['perimeter_mean'] / 100
X_test_scaled['area_mean'] = X_test_scaled['area_mean'] / 100
X_test_scaled['concavity_mean'] = X_test_scaled['concavity_mean'] / 100
X_test_scaled['concave points_mean'] = X_test_scaled['concave points_mean'] / 100

In [12]:
X_train_scaled.count()

radius_mean            455
perimeter_mean         455
area_mean              455
concavity_mean         455
concave points_mean    455
dtype: int64

In [13]:
y_train.count()

diagnosis    455
dtype: int64

In [14]:
model = keras.Sequential([
#     keras.layers.Dense(1, input_shape=(5,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros'),
    keras.layers.Dense(128, input_shape=(5,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=500)

Epoch 1/500
15/15 [==============================] - 2s 9ms/step - loss: 0.7354 - accuracy: 0.5604
Epoch 2/500
15/15 [==============================] - 0s 8ms/step - loss: 0.6765 - accuracy: 0.5824
Epoch 3/500
15/15 [==============================] - 0s 8ms/step - loss: 0.6902 - accuracy: 0.5758
Epoch 4/500
15/15 [==============================] - 0s 8ms/step - loss: 0.6838 - accuracy: 0.5692
Epoch 5/500
15/15 [==============================] - 0s 9ms/step - loss: 0.6800 - accuracy: 0.6132
Epoch 6/500
15/15 [==============================] - 0s 9ms/step - loss: 0.6784 - accuracy: 0.5890
Epoch 7/500
15/15 [==============================] - 0s 9ms/step - loss: 0.6971 - accuracy: 0.6242
Epoch 8/500
15/15 [==============================] - 0s 8ms/step - loss: 0.6902 - accuracy: 0.5429
Epoch 9/500
15/15 [==============================] - 0s 9ms/step - loss: 0.6727 - accuracy: 0.6198
Epoch 10/500
15/15 [==============================] - 0s 10ms/step - loss: 0.6699 - accuracy: 0.6198
Epoch 11

15/15 [==============================] - 0s 7ms/step - loss: 0.4509 - accuracy: 0.7934
Epoch 82/500
15/15 [==============================] - 0s 8ms/step - loss: 0.4212 - accuracy: 0.8022
Epoch 83/500
15/15 [==============================] - 0s 9ms/step - loss: 0.4151 - accuracy: 0.8242
Epoch 84/500
15/15 [==============================] - 0s 8ms/step - loss: 0.4181 - accuracy: 0.8264
Epoch 85/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3845 - accuracy: 0.8440
Epoch 86/500
15/15 [==============================] - 0s 8ms/step - loss: 0.4233 - accuracy: 0.8110
Epoch 87/500
15/15 [==============================] - 0s 7ms/step - loss: 0.4084 - accuracy: 0.8264
Epoch 88/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3947 - accuracy: 0.8286
Epoch 89/500
15/15 [==============================] - 0s 9ms/step - loss: 0.4380 - accuracy: 0.7912
Epoch 90/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3859 - accuracy: 0.8286
Epoch 91/500


15/15 [==============================] - 0s 9ms/step - loss: 0.3587 - accuracy: 0.8549
Epoch 163/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3608 - accuracy: 0.8549
Epoch 164/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3652 - accuracy: 0.8374
Epoch 165/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3923 - accuracy: 0.8396
Epoch 166/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3705 - accuracy: 0.8462
Epoch 167/500
15/15 [==============================] - 0s 10ms/step - loss: 0.3606 - accuracy: 0.8527
Epoch 168/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3800 - accuracy: 0.8418
Epoch 169/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3536 - accuracy: 0.8396
Epoch 170/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3479 - accuracy: 0.8440
Epoch 171/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3563 - accuracy: 0.8462
Epo

15/15 [==============================] - 0s 8ms/step - loss: 0.3501 - accuracy: 0.8505
Epoch 243/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3601 - accuracy: 0.8505
Epoch 244/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3535 - accuracy: 0.8505
Epoch 245/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3482 - accuracy: 0.8505
Epoch 246/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3502 - accuracy: 0.8571
Epoch 247/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3587 - accuracy: 0.8505
Epoch 248/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3397 - accuracy: 0.8527
Epoch 249/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3249 - accuracy: 0.8703
Epoch 250/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3688 - accuracy: 0.8484
Epoch 251/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3315 - accuracy: 0.8549
Epoc

15/15 [==============================] - 0s 8ms/step - loss: 0.3511 - accuracy: 0.8527
Epoch 324/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3594 - accuracy: 0.8418
Epoch 325/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3275 - accuracy: 0.8615
Epoch 326/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3491 - accuracy: 0.8352
Epoch 327/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3384 - accuracy: 0.8549
Epoch 328/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3465 - accuracy: 0.8484
Epoch 329/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3386 - accuracy: 0.8549
Epoch 330/500
15/15 [==============================] - 0s 10ms/step - loss: 0.3724 - accuracy: 0.8154
Epoch 331/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3597 - accuracy: 0.8484
Epoch 332/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3619 - accuracy: 0.8505
Epo

15/15 [==============================] - 0s 10ms/step - loss: 0.3436 - accuracy: 0.8549
Epoch 405/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3415 - accuracy: 0.8549
Epoch 406/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3348 - accuracy: 0.8549
Epoch 407/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3378 - accuracy: 0.8505
Epoch 408/500
15/15 [==============================] - 0s 10ms/step - loss: 0.3569 - accuracy: 0.8505
Epoch 409/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3444 - accuracy: 0.8681
Epoch 410/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3058 - accuracy: 0.8703
Epoch 411/500
15/15 [==============================] - 0s 7ms/step - loss: 0.3327 - accuracy: 0.8615
Epoch 412/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3397 - accuracy: 0.8549
Epoch 413/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3342 - accuracy: 0.8549
Ep

15/15 [==============================] - 0s 10ms/step - loss: 0.3401 - accuracy: 0.8637
Epoch 486/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3276 - accuracy: 0.8769
Epoch 487/500
15/15 [==============================] - 0s 8ms/step - loss: 0.3021 - accuracy: 0.8659
Epoch 488/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3161 - accuracy: 0.8484: 0s - loss: 0.3139 - accuracy: 0.85
Epoch 489/500
15/15 [==============================] - 0s 10ms/step - loss: 0.3398 - accuracy: 0.8681
Epoch 490/500
15/15 [==============================] - 0s 10ms/step - loss: 0.3160 - accuracy: 0.8637
Epoch 491/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3414 - accuracy: 0.8593
Epoch 492/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3386 - accuracy: 0.8703
Epoch 493/500
15/15 [==============================] - 0s 9ms/step - loss: 0.3192 - accuracy: 0.8440
Epoch 494/500
15/15 [==============================] - 0s 8ms/step

**Evaluation of the model on test set**

In [15]:
X_test_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114 entries, 522 to 360
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   radius_mean          114 non-null    float64
 1   perimeter_mean       114 non-null    float64
 2   area_mean            114 non-null    float64
 3   concavity_mean       114 non-null    float64
 4   concave points_mean  114 non-null    float64
dtypes: float64(5)
memory usage: 5.3 KB


In [16]:
X_test_scaled.count()

radius_mean            114
perimeter_mean         114
area_mean              114
concavity_mean         114
concave points_mean    114
dtype: int64

In [17]:
y_test.count()

diagnosis    114
dtype: int64

In [18]:
model.evaluate(X_test_scaled,y_test)

4/4 [==============================] - 0s 6ms/step - loss: 0.2924 - accuracy: 0.8860


[0.2924109995365143, 0.8859649300575256]

In [19]:
model.predict(X_test_scaled)

array([[0.05513288],
       [0.06265612],
       [0.12382036],
       [0.1482898 ],
       [0.25303638],
       [0.04888479],
       [0.01729202],
       [0.03087825],
       [0.05100284],
       [0.14487669],
       [0.09136198],
       [0.03099988],
       [0.4107543 ],
       [0.9987679 ],
       [0.9640114 ],
       [0.22693388],
       [0.98378736],
       [0.11097234],
       [0.07604646],
       [0.99407166],
       [0.255831  ],
       [0.42680517],
       [0.03416511],
       [0.99949837],
       [0.9996921 ],
       [0.99850523],
       [0.02350196],
       [0.2067884 ],
       [0.12744893],
       [0.9995548 ],
       [0.25126508],
       [0.02670613],
       [0.02715606],
       [0.64657843],
       [0.99090856],
       [0.99929094],
       [0.06096407],
       [0.02009077],
       [0.98605156],
       [0.418419  ],
       [0.08873986],
       [0.46120334],
       [0.34231   ],
       [0.40628764],
       [0.3415999 ],
       [0.18080327],
       [0.6470224 ],
       [0.445

**Value of weights and bias from the model**

In [ ]:
coef, intercept = model.get_weights()

In [ ]:
coef, intercept